<a href="https://colab.research.google.com/github/vitakurakina/ailabs/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22lab2_classification_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Лабораторная работа №2

## Задание
Решите пожалуйста следующую задачу.

Используйте классификатор SVM для данных находящихся пo
[ссылке](https://drive.google.com/file/d/1LBDnhITL0Wqwp5G6M6IBI-SSz8BIoNec/view?usp=share_link) (данный датасет применялся в лабораторной работе №1). Здесь - выходящей меткой (учителем) является столбец / атрибут - `outcome`. Предоставьте результаты, используя следующие ядерные функции:
- Линейное ядро
- Полиномиальное ядро
- Гауссово ядро
- Сигмоидное ядро

**Укажите `score` моделей.**

## Загрузка данных

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Разделение данных

### Subtask:
Разделите данные на обучающий и тестовый наборы.

**Reasoning**:
The subtask is to split the data into training and testing sets. This requires importing the necessary function and then applying it to the dataframe.

In [ ]:
! gdown --fuzzy 'https://drive.google.com/file/d/1LBDnhITL0Wqwp5G6M6IBI-SSz8BIoNec/view?usp=share_link'

Downloading...
From: https://drive.google.com/uc?id=1LBDnhITL0Wqwp5G6M6IBI-SSz8BIoNec
To: /content/diabetes.csv
100% 23.1k/23.1k [00:00<00:00, 41.0MB/s]


In [ ]:
df = pd.read_csv('/content/diabetes.csv')
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


##  О датасете
[Pima Indians Diabetes Database](https://www.kaggle.com/datasets/uciml/pima-indians-diabetes-database/data)

Этот набор данных взят из Национального института диабета, заболеваний органов пищеварения и почек (NIDDK). Целью набора данных является диагностическое предсказание наличия у пациента диабета на основе определенных диагностических измерений, включенных в набор данных. На выбор этих экземпляров из более крупной базы данных было наложено несколько ограничений. В частности, все пациенты здесь — женщины не моложе 21 года, родом из индейцев Пима.

Наборы данных состоят из нескольких переменных медицинских предикторов и одной целевой переменной `Outcome`. Переменные-предикторы включают количество беременностей, которые были у пациентки, её индекс массы тела `BMI`, уровень инсулина, возраст и так далее. Всего в датасете 9 переменных:
- `Pregnancies` - количество беременностей;
- `Glucose` - концентрация глюкозы в плазме через 2 часа при пероральном тесте на толерантность к глюкозе;
- `BloodPressure` - диастолическое артериальное давление (мм рт. ст.);
- `SkinThickness` - толщина кожной складки трицепса (мм);
- `Insulin` - 2-часовой сывороточный инсулин (мкЕд/мл);
- `BMI` - Индекс массы тела (вес в кг/(рост в м)^2);
- `DiabetesPedigreeFunction` - функция оценки вероятности диабета на основе семейного анамнеза;
- `Age` - возраст (лет);
- `Outcome` - целевая переменная (0 или 1) 268 из 768 — 1 (больны), остальные — 0 (здоровы).



In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [ ]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Pregnancies,768.0,3.845052,3.369578,0.000,1.00000,3.0000,6.00000,17.00
Glucose,768.0,120.894531,31.972618,0.000,99.00000,117.0000,140.25000,199.00
BloodPressure,768.0,69.105469,19.355807,0.000,62.00000,72.0000,80.00000,122.00
SkinThickness,768.0,20.536458,15.952218,0.000,0.00000,23.0000,32.00000,99.00
Insulin,768.0,79.799479,115.244002,0.000,0.00000,30.5000,127.25000,846.00
BMI,768.0,31.992578,7.884160,0.000,27.30000,32.0000,36.60000,67.10
DiabetesPedigreeFunction,768.0,0.471876,0.331329,0.078,0.24375,0.3725,0.62625,2.42
Age,768.0,33.240885,11.760232,21.000,24.00000,29.0000,41.00000,81.00
Outcome,768.0,0.348958,0.476951,0.000,0.00000,0.0000,1.00000,1.00


In [ ]:
df.columns

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'],
      dtype='object')

## Классификатор SVM

In [ ]:
from sklearn.model_selection import train_test_split

X = df.drop('Outcome', axis=1)
y = df['Outcome']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Масштабирование данных

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

### Линейное ядро

In [ ]:
from sklearn.svm import SVC

svm_linear = SVC(kernel='linear')
svm_linear.fit(X_train, y_train)

SVC(kernel='linear')

### Полиномиальное ядро

In [ ]:
from sklearn.svm import SVC

svm_poly = SVC(kernel='poly')
svm_poly.fit(X_train, y_train)

SVC(kernel='poly')

### Гауссово ядро

In [ ]:
from sklearn.svm import SVC

svm_rbf = SVC(kernel='rbf')
svm_rbf.fit(X_train, y_train)

SVC()

### Сигмоидное ядро

In [ ]:
from sklearn.svm import SVC

svm_sigmoid = SVC(kernel='sigmoid')
svm_sigmoid.fit(X_train, y_train)

SVC(kernel='sigmoid')

# Task
Apply SVM classifier with different kernels (linear, polynomial, RBF, sigmoid) to the data, evaluate each model, and output the scores.

## Разделение данных

### Subtask:
Разделите данные на обучающий и тестовый наборы.


**Reasoning**:
The subtask is to split the data into training and testing sets. This requires importing the necessary function and then applying it to the dataframe.



In [ ]:
from sklearn.model_selection import train_test_split

X = df.drop('Outcome', axis=1)
y = df['Outcome']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
score_linear = svm_linear.score(X_test, y_test)
score_poly = svm_poly.score(X_test, y_test)
score_rbf = svm_rbf.score(X_test, y_test)
score_sigmoid = svm_sigmoid.score(X_test, y_test)

print(f"Score for Linear Kernel: {score_linear}")
print(f"Score for Polynomial Kernel: {score_poly}")
print(f"Score for Gaussian Kernel (RBF): {score_rbf}")
print(f"Score for Sigmoid Kernel: {score_sigmoid}")

Score for Linear Kernel: 0.7597402597402597
Score for Polynomial Kernel: 0.7467532467532467
Score for Gaussian Kernel (RBF): 0.7337662337662337
Score for Sigmoid Kernel: 0.6428571428571429


**Reasoning**:
The error indicates that the dataframe `df` is not defined. Looking at the notebook state, the cells that define `df` have not been executed in the current runtime. I need to re-execute the code cell that loads the data into the dataframe.



In [ ]:
df = pd.read_csv('/content/diabetes.csv')
from sklearn.model_selection import train_test_split

X = df.drop('Outcome', axis=1)
y = df['Outcome']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

**Reasoning**:
The file '/content/diabetes.csv' is not found. Looking at the notebook state, the cell with `! gdown` command has not been executed in the current runtime. I need to execute the cell that downloads the file and then the cell that reads the file into the dataframe.



In [ ]:
! gdown --fuzzy 'https://drive.google.com/file/d/1LBDnhITL0Wqwp5G6M6IBI-SSz8BIoNec/view?usp=share_link'
df = pd.read_csv('/content/diabetes.csv')
from sklearn.model_selection import train_test_split

X = df.drop('Outcome', axis=1)
y = df['Outcome']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Downloading...
From: https://drive.google.com/uc?id=1LBDnhITL0Wqwp5G6M6IBI-SSz8BIoNec
To: /content/diabetes.csv
100% 23.1k/23.1k [00:00<00:00, 58.4MB/s]


## Масштабирование данных

### Subtask:
Примените масштабирование признаков к данным.


**Reasoning**:
Apply StandardScaler to the training and testing data as instructed.



In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## Применение svm с различными ядрами

### Subtask:
Обучите модели SVM с линейным, полиномиальным, гауссовым и сигмоидным ядрами.


**Reasoning**:
Import SVC and train SVM models with different kernels.



In [ ]:
from sklearn.svm import SVC

svm_linear = SVC(kernel='linear')
svm_linear.fit(X_train, y_train)

svm_poly = SVC(kernel='poly')
svm_poly.fit(X_train, y_train)

svm_rbf = SVC(kernel='rbf')
svm_rbf.fit(X_train, y_train)

svm_sigmoid = SVC(kernel='sigmoid')
svm_sigmoid.fit(X_train, y_train)

SVC(kernel='sigmoid')

## Оценка моделей

### Subtask:
Вычислите и выведите `score` для каждой модели.


**Reasoning**:
Compute and print the scores for each trained SVM model using the test data.



In [ ]:
score_linear = svm_linear.score(X_test, y_test)
score_poly = svm_poly.score(X_test, y_test)
score_rbf = svm_rbf.score(X_test, y_test)
score_sigmoid = svm_sigmoid.score(X_test, y_test)

print(f"Score for Linear Kernel: {score_linear}")
print(f"Score for Polynomial Kernel: {score_poly}")
print(f"Score for Gaussian Kernel (RBF): {score_rbf}")
print(f"Score for Sigmoid Kernel: {score_sigmoid}")

Score for Linear Kernel: 0.7597402597402597
Score for Polynomial Kernel: 0.7467532467532467
Score for Gaussian Kernel (RBF): 0.7337662337662337
Score for Sigmoid Kernel: 0.6428571428571429


## Анализ результатов

На основе выполненных задач и подзадач были получены следующие результаты:

### Ключевые выводы из анализа данных

*   Набор данных был успешно загружен и изучен с помощью базовых функций pandas (`info()`, `describe()`).
*   Данные были разделены на обучающую (80%) и тестовую (20%) выборки.
*   Было выполнено масштабирование признаков с использованием `StandardScaler` для обучающего и тестового наборов данных.
*   Были обучены четыре модели SVM с различными ядерными функциями: линейной, полиномиальной, гауссовой (RBF) и сигмоидной.
*   Оценки (`score`) производительности моделей на тестовой выборке составили:
    *   Линейное ядро: {{score_linear:.4f}}
    *   Полиномиальное ядро: {{score_poly:.4f}}
    *   Гауссово ядро (RBF): {{score_rbf:.4f}}
    *   Сигмоидное ядро: {{score_sigmoid:.4f}}

### Интерпретация результатов и дальнейшие шаги

*   Среди протестированных ядер, линейное ядро показало наилучший результат на данном наборе данных. Это может говорить о том, что данные имеют относительно линейно разделимую структуру в признаковом пространстве после масштабирования.
*   Производительность полиномиального и гауссова ядер была близка к линейному, в то время как сигмоидное ядро показало значительно более низкий результат.
*   Для потенциального улучшения производительности моделей с полиномиальным, гауссовым и сигмоидным ядрами можно провести подбор гиперпараметров (`C`, `gamma`, `degree` для полиномиального ядра) с использованием кросс-валидации.
*   Также можно рассмотреть другие метрики оценки классификации, помимо `score` (точность), такие как полнота (recall), прецизионность (precision), F1-мера и ROC-кривая, для более полного понимания производительности каждой модели, особенно учитывая, что классы в данных могут быть несбалансированы.

## Summary:

### Data Analysis Key Findings

*   The dataset was successfully split into training (80%) and testing (20%) sets.
*   Feature scaling was applied to both the training and testing datasets using `StandardScaler`.
*   Four SVM models were trained using different kernels: linear, polynomial, RBF (Gaussian), and sigmoid.
*   The evaluation scores on the test set for each kernel were:
    *   Linear Kernel: 0.7597
    *   Polynomial Kernel: 0.7468
    *   Gaussian Kernel (RBF): 0.7338
    *   Sigmoid Kernel: 0.6429

### Insights or Next Steps

*   The linear kernel achieved the highest score among the tested kernels on this dataset, suggesting a relatively linear separation boundary might exist.
*   Further hyperparameter tuning for each kernel, especially 'poly', 'rbf', and 'sigmoid', could potentially improve their performance.
